In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import functions as f
from pyspark.sql.functions import col

spark=SparkSession.builder.appName("Dunhumbby Practise").getOrCreate()
#create DataFrame with movie CSV Data
rawDFMovie=spark.read.format("CSV").option("header","true").option("inferSchema","true").load("/FileStore/tables/movies.csv")
rawDFRatings=spark.read.format("CSV").option("header","true").option("inferSchema","true").load("/FileStore/tables/ratings.csv")
rawDFLinks=spark.read.format("CSV").option("header","true").option("inferSchema","true").load("/FileStore/tables/links.csv")
rawDFTags=spark.read.format("CSV").option("header","true").option("inferSchema","true").load("/FileStore/tables/tags.csv")

## Persist Data
rawDFMovie.persist()
rawDFRatings.persist()
rawDFLinks.persist()
rawDFTags.persist()

Out[ 1 ]: DataFrame[userId: int, movieId: int, tag: string, timestamp: int]

In [2]:
#Print Column Details
print(rawDFMovie.dtypes)
print(rawDFRatings.dtypes)
print(rawDFLinks.dtypes)
print(rawDFTags.dtypes)


[('movieId', 'int'), ('title', 'string'), ('genres', 'string')]
[('userId', 'int'), ('movieId', 'int'), ('rating', 'double'), ('timestamp', 'int')]
[('movieId', 'int'), ('imdbId', 'int'), ('tmdbId', 'int')]
[('userId', 'int'), ('movieId', 'int'), ('tag', 'string'), ('timestamp', 'int')]

In [3]:
#number of unique movies
rawDFMovie.select("movieId").count()

Out[ 3 ]: 27278

In [4]:
#minimum rating given to a movie
minRating=rawDFRatings.groupBy().agg(f.min("rating").alias("MinRating")).rdd.first().MinRating
print(minRating)

##minimum rating given to a movie with movieID
movieIDminRate=rawDFRatings.filter(col("rating")==minRating)


0.5

In [5]:
#movie Details with min rating
movieIDminRate.alias('a').join(rawDFMovie.alias('b'), "movieID").select("movieID","rating","title","genres").show(2)

+-------+------+--------------------+--------------------+
movieID|rating| title| genres|
+-------+------+--------------------+--------------------+
 286| 0.5|Nemesis 2: Nebula...|Action|Sci-Fi|Thr...|
 671| 0.5|Mystery Science T...| Comedy|Sci-Fi|
+-------+------+--------------------+--------------------+
only showing top 2 rows

In [6]:
#movies with min average rating
avgRatingDF=rawDFRatings.groupBy(rawDFRatings["movieID"]).agg(f.avg(col("rating")).alias("avg_rating"))
minAvgRating=avgRatingDF.select("avg_rating").distinct().agg(f.min(col("avg_rating")).alias("minAvgRating")).rdd.first().minAvgRating
avgRatingDF.filter(col("avg_rating")==minAvgRating).join(rawDFMovie.alias('b'), "movieID").select("movieID","avg_rating","title","genres").show(2)



+-------+----------+--------------------+-----------+
movieID|avg_rating| title| genres|
+-------+----------+--------------------+-----------+
 7281| 0.5|Scream, Blacula, ...| Horror|
 8716| 0.5|Plainsman, The (1...|War|Western|
+-------+----------+--------------------+-----------+
only showing top 2 rows

 Out[ 33 ]: <bound method DataFrame.printSchema of DataFrame[movieId: int, userId: int, rating: double, timestamp: int, title: string, genres: string]>

In [7]:
#create Join DF for further analytics
movie_ratingDF=rawDFRatings.join(f.broadcast(rawDFMovie), "movieID").select("movieId","rating","title","genres")
movie_ratingDF.persist()

Out[ 40 ]: DataFrame[movieId: int, rating: double, title: string, genres: string]

In [8]:
#moast popular genre
avgRating_genre_DF=movie_ratingDF.groupBy(movie_ratingDF["genres"]).agg(f.avg(col("rating")).alias("avg_rating"))
avgRating_genre_DF.show(2)
##******************************************************************************************************************************************************##


#Best Movie Per Year avgRatingDF=rawDFRatings.groupBy(rawDFRatings["movieID"]).agg(f.avg(col("rating")).alias("avg_rating"))movie with max average rating per year

+--------------------+------------------+
 genres| avg_rating|
+--------------------+------------------+
Comedy|Horror|Thr...|3.2884195193008012|
Action|Adventure|...| 4.074866310160428|
+--------------------+------------------+
only showing top 2 rows

In [9]:
#unique tag

#Best 2 movie with each tag

#Best 2 movie with 

In [10]:
# filtering to get the list of drama movies

#total number of drama movies in year 1995


In [11]:
#movie ID and Name with tag Dark

In [12]:
#displays high rated movies (rating>4)

#displays high rated movies (rating<4)

In [13]:
#top 25 most rated movies


In [14]:
#most popular genere